In [7]:
import pandas as pd
import math
import numpy as np
import json

from pathlib import Path
import scipy.stats as st

In [8]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "val_log.csv").is_file():
        with open(run / 'config.json') as f:
            cfg = json.load(f)

        num_epochs = cfg['num_epochs']
        val_iter = cfg['validate_iter']
        num_epochs = int(num_epochs / val_iter)

        valid_log = pd.read_csv(run / "val_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-1) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("val_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    with open(run / 'config.json') as f:
        cfg = json.load(f)

    run_number = cfg['seed']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = int(run_number)
    data['inv_temp'] = int(inv_temp)
    data['regime'] = int(regime)
    
    return data

def collect_all(model_name, root, csv_file, regimes=['1', '2', '5', '10', '20','100'], ignore_outliers=True):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    if ignore_outliers and 'outlier' in run.as_posix():
                        continue
                    else:
                        metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [9]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], int(model_group[1]), int(model_group[2]), int(model_group[3])
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90, return_ranges=False):

    def compute_ci(values, return_ranges=False):
        ci = st.t.interval(confidence_level, len(values)-1, loc=np.mean(values), scale=np.std(values)/(len(values)**0.5))

        if return_ranges:
            return ci
        else:
            return (ci[1]-ci[0]) / 2

    #mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', 'mean'), ("CI {}%".format(confidence_level), compute_ci)]) 

    return mean_metrics

<h1>Atrial Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [ ]:
EXP_ROOT = "./runs"

REGIMES = ['100']

runs = {
    'H-UNet3D-SWTA-T': list(Path(EXP_ROOT + '/Atrial/hebbian_unsup/').glob('unet3d_swta_t')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'])

display(summary)

In [72]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'H-UNet3D-SWTA-T': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('h_unet3d_swta_t')),
    #'H-EM-SWTA-T': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('h_em_unet3d_swta_t')),
}

In [73]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [74]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet3D-SWTA-T,0,1,1,36.85,22.59,48.04,13.04
1,H-UNet3D-SWTA-T,0,1,2,38.03,23.48,48.26,13.62
2,H-UNet3D-SWTA-T,0,1,5,40.47,25.37,47.35,12.13
3,H-UNet3D-SWTA-T,0,1,10,40.88,25.69,46.49,11.62
4,H-UNet3D-SWTA-T,0,1,20,42.63,27.09,47.35,11.44
...,...,...,...,...,...,...,...,...
170,H-UNet3D-SWTA-T,4,100,1,60.54,43.41,30.53,8.62
171,H-UNet3D-SWTA-T,4,100,2,62.79,45.76,33.42,8.59
172,H-UNet3D-SWTA-T,4,100,5,79.29,65.68,20.75,5.29
173,H-UNet3D-SWTA-T,4,100,10,86.21,75.75,13.95,3.88


Dice           Jaccard            \
                                   Mean   CI 0.9%    Mean   CI 0.9%   
Model           Inv Temp Regime                                       
H-UNet3D-SWTA-T 1        1       34.342  4.932441  20.846  3.448707   
                         2       38.304  1.767018  23.706  1.339093   
                         5       38.098  1.525895  23.544  1.166206   
                         10      39.846  1.371685  24.890  1.068274   
                         20      43.866  1.379245  28.108  1.136455   
                5        1       29.796  2.989640  17.548  2.100071   
                         2       39.262  2.562089  24.462  2.003731   
                         5       40.656  1.736571  25.532  1.376710   
                         10      42.816  3.606614  27.316  2.950258   
                         20      46.264  3.395537  30.168  2.902466   
                10       1       29.778  3.360707  17.548  2.331055   
                         2       39.434  2.914294  24.608  2.266404   
                         5       43.212  1.715885  27.578  1.389512   
                         10      46.274  1.300274  30.112  1.103719   
                         20      51.094  1.305841  34.324  1.172238   
                20       1       49.674  3.411129  33.122  3.092218   
                         2       57.514  7.072098  40.740  6.840482   
                         5       71.042  2.811681  55.172  3.415814   
                         10      78.326  5.589439  64.738  7.313191   
                         20      83.168  2.773141  71.294  4.189428   
                50       1       59.360  4.694569  42.384  4.825200   
                         2       74.066  4.689189  59.040  5.665453   
                         5       81.860  2.010475  69.342  2.886073   
                         10      84.926  1.961509  73.852  2.955969   
                         20      88.598  0.496181  79.528  0.800630   
                75       1       54.088  8.417113  37.566  7.868266   
                         2       71.544  3.262646  55.804  3.902272   
                         5       81.634  2.091789  69.024  2.969304   
                         10      84.870  2.431719  73.800  3.586894   
                         20      88.120  0.354272  78.764  0.561792   
                100      1       57.708  6.241900  40.846  6.031270   
                         2       72.876  5.313084  57.618  6.351602   
                         5       80.476  2.976282  67.438  4.087677   
                         10      84.770  2.714584  73.664  3.980905   
                         20      88.264  0.359902  78.998  0.576511   

                                Hausdorff Distance            \
                                              Mean   CI 0.9%   
Model           Inv Temp Regime                                
H-UNet3D-SWTA-T 1        1                  49.088  2.586818   
                         2                  47.786  0.548729   
                         5                  48.372  1.343969   
                         10                 47.740  0.697580   
                         20                 47.280  0.356369   
                5        1                  54.752  1.304752   
                         2                  50.582  2.513623   
                         5                  50.842  2.610567   
                         10                 50.222  3.700952   
                         20                 48.754  4.065336   
                10       1                  54.816  1.071615   
                         2                  51.520  1.055314   
                         5                  49.240  2.217244   
                         10                 47.372  1.310660   
                         20                 45.272  1.387325   
                20       1                  44.222  7.028971   
                         2                  39.584  6.222801   
                         5                  28.526  2.98

<h2>Evaluation - Weight init comparison</h2>

<p>Evaluate weight init methods (only for baseline)</p>

In [43]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'Kaiming-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_unet3d')),
    'Xavier-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('xavier_unet3d')),
    'Orthogonal-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('orthogonal_unet3d')),
}

In [44]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90)

display(summary)

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset.</p>

In [10]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_unet3d')),
    'VNet': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('kaiming_vnet')),
    'EM-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('em_unet3d')),
    'CPS-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('cps_unet3d')),
    'UAMT-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('uamt_unet3d')),
    'URPC-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('urpc_unet3d')),
    'CCT-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('cct_unet3d')),
    'DTC-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('dtc_unet3d')),
}

In [11]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90)

display(summary)

<h1>LiTS Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

<h2>Evaluation - Weight init comparison</h2>

<p>Evaluate weight init methods (only for baseline)</p>

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset.</p>

In [18]:
EXP_ROOT = "./runs"

REGIMES = ['100']

runs = {
    'UNet3D': list(Path(EXP_ROOT + '/LiTS/fully_sup/').glob('unet3d')),
    #'VNet': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_vnet')),
    #'EM-UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_em_unet3d')),
    #'CPS-UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_cps_unet3d')),
    #'UAMT-UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_uamt_unet3d')),
    #'URPC-UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_urpc_unet3d')),
    #'CCT-UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_cct_unet3d')),
    #'DTC-UNet3D': list(Path(EXP_ROOT + '/LiTS/semi_sup/').glob('kaiming_dtc_unet3d')),
}

In [19]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [20]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,UNet3D,0,1,100,82.12,73.49,34.10,11.62
1,UNet3D,1,1,100,81.39,72.44,43.91,12.43


Dice           Jaccard           Hausdorff Distance  \
                          Mean   CI 0.9%    Mean   CI 0.9%               Mean   
Model  Inv Temp Regime                                                          
UNet3D 1        100     81.755  1.629541  72.965  2.343861             39.005   

                                  Average Surface Distance            
                          CI 0.9%                     Mean   CI 0.9%  
Model  Inv Temp Regime                                                
UNet3D 1        100     21.898355                   12.025  1.808121